In [10]:
import pandas as pd
# convert txt file to csv
# the text file  sparted by ; and csv should be  ,
# the col in top
read_file = pd.read_csv('train.txt')
read_file.to_csv('train.csv', index=None)


In [12]:
from pandas import read_csv
df = pd.read_csv('train.csv')
df.shape


(16000, 2)

In [13]:
# check the count for emotions
df['Emotion'].value_counts()


Emotion
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

In [15]:
df['Emotion_number'] = df['Emotion'].map({
    'joy': 0,
    'sadness': 1,
    'anger': 2,
    'fear': 3,
    'love': 4,
    'surprise': 5,
})

df.head()


,Comment,Emotion,Emotion_number
0,i didnt feel humiliated,sadness,1
1,i can go from feeling so hopeless to so damned...,sadness,1
2,im grabbing a minute to post i feel greedy wrong,anger,2
3,i am ever feeling nostalgic about the fireplac...,love,4
4,i am feeling grouchy,anger,2


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Comment,
                                                    df.Emotion_number,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=df.Emotion_number)

X_train.shape, X_test.shape


((12800,), (3200,))

In [18]:
# first try using random forest
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

clf = Pipeline([
    ('vectorizer_trie_gram', CountVectorizer(ngram_range=(3, 3))),
    ('random_forest', RandomForestClassifier())
])

clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)

print(classification_report(y_test, y_predict))


              precision    recall  f1-score   support

           0       0.42      0.83      0.55      1072
           1       0.53      0.37      0.43       933
           2       0.53      0.17      0.26       432
           3       0.48      0.26      0.33       387
           4       0.43      0.08      0.14       261
           5       0.53      0.07      0.12       115

    accuracy                           0.45      3200
   macro avg       0.49      0.30      0.31      3200
weighted avg       0.48      0.45      0.40      3200



In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('random_forest', RandomForestClassifier()),
])

clf.fit(X_train, y_train)

y_predict = clf.predict(X_test)

print(classification_report(y_test, y_predict))


              precision    recall  f1-score   support

           0       0.80      0.93      0.86      1072
           1       0.91      0.87      0.89       933
           2       0.91      0.80      0.85       432
           3       0.82      0.80      0.81       387
           4       0.85      0.70      0.77       261
           5       0.79      0.69      0.73       115

    accuracy                           0.85      3200
   macro avg       0.85      0.80      0.82      3200
weighted avg       0.85      0.85      0.85      3200



In [21]:
import spacy

nlp = spacy.load("en_core_web_sm")


def preprocess(text):
    return " ".join([token.lemma_ for token in nlp(text) if not token.is_stop and not token.is_punct])


In [22]:
# this will take some time, please be patient
df['preprocessed_comment'] = df['Comment'].apply(preprocess)


In [23]:
df.head()


,Comment,Emotion,Emotion_number,preprocessed_comment
0,i didnt feel humiliated,sadness,1,not feel humiliate
1,i can go from feeling so hopeless to so damned...,sadness,1,feel hopeless damned hopeful care awake
2,im grabbing a minute to post i feel greedy wrong,anger,2,m grab minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,love,4,feel nostalgic fireplace know property
4,i am feeling grouchy,anger,2,feel grouchy


In [24]:
from sympy.abc import y
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.preprocessed_comment, df.Emotion_number, test_size=0.2, random_state=2023, stratify=df.Emotion_number)


In [25]:
# 1. create a pipeline object
clf = Pipeline([
    # using the ngram_range parameter
    ('vectorizer_bi_grams', CountVectorizer(ngram_range=(1, 2))),
    ('random_forest', (RandomForestClassifier()))
])

# 2. fit with X_train and y_train
clf.fit(X_train, y_train)


# 3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


# 4. print the classfication report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1072
           1       0.88      0.93      0.90       933
           2       0.89      0.86      0.88       432
           3       0.91      0.80      0.86       387
           4       0.78      0.70      0.74       261
           5       0.78      0.77      0.77       115

    accuracy                           0.88      3200
   macro avg       0.86      0.83      0.84      3200
weighted avg       0.88      0.88      0.87      3200



In [26]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),  # using the ngram_range parameter
    ('Random Forest', RandomForestClassifier())
])

# 2. fit with X_train and y_train
clf.fit(X_train, y_train)


# 3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


# 4. print the classfication report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.81      0.92      0.86      1072
           1       0.90      0.87      0.88       933
           2       0.90      0.80      0.85       432
           3       0.82      0.80      0.81       387
           4       0.76      0.65      0.70       261
           5       0.78      0.63      0.70       115

    accuracy                           0.84      3200
   macro avg       0.83      0.78      0.80      3200
weighted avg       0.84      0.84      0.84      3200

